In diesem Notebook wird die Pipeline aufgebaut.

In [1]:
%load_ext autoreload
%autoreload 2

import os
if os.getcwd() == '/home/jovyan/work': # jhub
    os.chdir("24FS_I4DS27/main/") 
    os.system("make reqs")
else: # local
    os.chdir("../")

In [2]:
import numpy as np
import torch
import torchvision
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams["figure.dpi"] = 200
plt.rcParams["figure.figsize"] = (16, 8)

In [10]:
from lightning.pytorch import loggers as pl_loggers

# create logger
logger = pl_loggers.CSVLogger("logs", name="test", version=0)

# log hyperparameters
logger.log_hyperparams({"lr": 0.01, "bs": 32})

# log metrics
for i in range(50):
    logger.log_metrics({"loss": 1.0 / (i + 1), "acc": 0.5 * (i + 1)}, step=i)

# print log directory
logger.save()